In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Direct Adjustments

Attributes
----------
DirectAdjustment
Transactions
Portfolio
Instruments
"""

toggle_code("Toggle Docstring")

# Direct Adjustments

This notebook demonstrates how to create transactions that update the units of an holding without having any impact on the cost.

In this notebook, we will:
1. **[Setup LUSID](#1.-Setup-LUSID)**
2. **[Load Data](#2.-Load-Data)**
3. **[Create Portfolio](#3.-Create-Portfolio)**
4. **[Load Instrument](#4.-Load-Instrument)**
5. **[Load Transactions](#5.-Load-Transactions)**
6. **[Configure Transaction Types](#6.-Configure-Transaction-Types)**
7. **[Get Holdings](#7.-Get-Holdings)**

## 1. Setup LUSID
Initialise our LUSID environment.

In [2]:
import json
import lusid
import pandas as pd
from lusid import models
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    app_name="LusidJupyterNotebook",
)

print("LUSID Environment Initialised")
print(
    "LUSID SDK Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID SDK Version:  0.6.11054.0


In [3]:
# Build the APIs
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
transaction_configuration_api = api_factory.build(lusid.api.TransactionConfigurationApi)

## 2. Load Data

Pull data from our local csv data file. Note the `DirectAdjustmentIncrease` and `DirectAdjustmentDecrease` transaction type, once we create them, it is these that will allow an increase/decrease in units to occur without impacting the cost of the transaction.

For these transactions, the price will need to be set to 0.

In [4]:
txns = pd.read_csv("./data/transactions.csv")
txns

,txn_id,trade_date,transaction_type,instrument_desc,client_internal,instrument_currency,quantity,price,net_money
0,txn01,2020-01-01T00:00:00.0000000+00:00,FundsIn,CASH_GBP,cash,GBP,1000000,1.00,1000000
1,txn02,2020-01-02T00:00:00.0000000+00:00,Buy,Tesco,EQ_001,GBP,100000,2.56,256000
2,txn03,2020-01-03T00:00:00.0000000+00:00,Sell,Tesco,EQ_001,GBP,30000,2.56,256000
3,txn04,2020-01-04T00:00:00.0000000+00:00,DirectAdjustmentIncrease,Tesco,EQ_001,GBP,50000,0.00,0
4,txn05,2020-01-05T00:00:00.0000000+00:00,DirectAdjustmentDecrease,Tesco,EQ_001,GBP,40000,0.00,0


## 3. Create Portfolio

Create a portfolio to upload our transactions into.

In [5]:
# Scope & code
scope = "DirectAdjustmentsExample"
portfolio_code = "UK_EQUITY"

# Create portfolio with properties
subholding_key = {}
created_date = "2010-01-01T00:00:00.000000+00:00"

# create request body
portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name="UK-EQUITY",
    code=portfolio_code,
    base_currency="GBP",
    created=created_date
)

# Upload new portfolio to LUSID
try:
    response = transaction_portfolios_api.create_portfolio(
        scope=scope, create_transaction_portfolio_request=portfolio_request
    )

    print(f"Portfolio '{response.id.code}' created")
    
except lusid.ApiException as e:
    if json.loads(e.body)["code"] == 112: # PortfolioWithIdAlreadyExists
        print(json.loads(e.body)["title"])
    else:
        raise e

Portfolio 'UK_EQUITY' created


## 4. Load Instrument

Create instrument for `Tesco`.

In [6]:
# Remove duplicate values
instrument_unique = txns.drop_duplicates(subset="client_internal")

# Remove cash rows
instruments = instrument_unique[instrument_unique["client_internal"] != "cash"]

request_body = {
    instr["instrument_desc"]: models.InstrumentDefinition(
        name=instr["instrument_desc"],
        identifiers={ "ClientInternal": models.InstrumentIdValue(value=instr["client_internal"]) },
    )
    for row, instr in instruments.iterrows()
}

# Upsert new instruments to LUSID
instrument_response = instruments_api.upsert_instruments(
    request_body=request_body
)

# Check response was successful
if len(instrument_response.failed) > 0:
    raise AssertionError("Instruments upsert failed. Inspect response for more detail")
else:
    val_len = len(instrument_response.values)
    print(f"{val_len} {'instrument' if val_len == 1 else 'instruments'} created")

1 instrument created


## 5. Load Transactions

Load in our transactions from the data file.

In [7]:
# Upsert transactions
transactions_request = []
txn_response = []
txn_type_source = "testsource"
for row, txn in txns.iterrows():

    if txn["client_internal"] == "cash":
        instrument_identifier = {"Instrument/default/Currency": "GBP"}
    else:
        instrument_identifier = {
            "Instrument/default/ClientInternal": txn["client_internal"]
        }

    # Build request body
    transactions_request.append(
        models.TransactionRequest(
            transaction_id=txn["txn_id"],
            type=txn["transaction_type"],
            instrument_identifiers=instrument_identifier,
            transaction_date=txn["trade_date"],
            settlement_date=txn["trade_date"],
            units=txn["quantity"],
            transaction_price=models.TransactionPrice(price=txn["price"], type="Price"),
            total_consideration=models.CurrencyAndAmount(
                amount=txn["net_money"], currency=txn["instrument_currency"]
            ),
            source=txn_type_source
        )
    )

    # Make upsert transactions call to LUSID
    txn_response.append(
        transaction_portfolios_api.upsert_transactions(
            scope=scope, code=portfolio_code, transaction_request=transactions_request
        )
    )

print(f"{len(txn_response)} {'transaction' if len(txn_response) == 1 else 'transactions'} upserted")

5 transactions upserted


## 6. Configure Transaction Types

In this section we configure some new transaction types.

`DirectAdjustmentDecrease` to **decrease** units without an impact on cost.

`DirectAdjustmentIncrease` to **increase** units without an impact on cost.

First, let's create the sides.

In [8]:
# Define our side
response = transaction_configuration_api.set_side_definition(
    "Tutorial-Side1", 
    side_definition_request=models.SideDefinitionRequest(
        security="Txn:LusidInstrumentId",
        currency="Txn:TradeCurrency",
        rate="Txn:TradeToPortfolioRate",
        units="Txn:Units",
        amount="Txn:TradeAmount"
    )
)

print(f"'{response.side}' has been created in LUSID")

'Tutorial-Side1' has been created in LUSID


This outcome we are trying to achieve is contingent on the `movement_options` field within our new transaction type being set to `DirectAdjustment`. Without this, we will not be able to adjust our unit values without any impact on the cost.

In [9]:
types = [
    (
        "DirectAdjustmentDecrease",
         models.TransactionTypeRequest(
             aliases=[
                 models.TransactionTypeAlias(
                     type="DirectAdjustmentDecrease",
                     description="A negative direct adjustment type",
                     transaction_class="Custom",
                     transaction_roles="ShortShorter",
                 )
             ],
             movements=[
                 models.TransactionTypeMovement(
                     movement_types="StockMovement",
                     side="Tutorial-Side1",
                     direction=-1,
                     properties={},
                     mappings=[],
                     movement_options=["DirectAdjustment"] # Value required
                 )
             ]
         )
    ),
    (
        "DirectAdjustmentIncrease",
        models.TransactionTypeRequest(
             aliases=[
                 models.TransactionTypeAlias(
                     type="DirectAdjustmentIncrease",
                     description="A positive direct adjustment type",
                     transaction_class="Custom",
                     transaction_roles="LongLonger",
                 )
             ],
             movements=[
                 models.TransactionTypeMovement(
                     movement_types="StockMovement",
                     side="Tutorial-Side1",
                     direction=1,
                     properties={},
                     mappings=[],
                     movement_options=["DirectAdjustment"] # Value required
                 )
             ]
         )
    )
]

try:
    for type in types:
        response = transaction_configuration_api.set_transaction_type(
            source=txn_type_source,
            type=type[0],
            transaction_type_request=type[1]
        )
        print(f"Transaction type '{response.aliases[0].type}' created")

except lusid.ApiException as e:
    print(json.loads(e.body))


Transaction type 'DirectAdjustmentDecrease' created
Transaction type 'DirectAdjustmentIncrease' created


## 7. Get Holdings

To verify the `DirectAdjustment` transaction types have had the desired effect, let's fetch the holdings from points in time before and after the transactions have settled.

In [10]:
# Prints the name and a quick summary from a get_holdings() response
def display_holdings_summary(response):
    # Inspect holdings response for today
    hld = [i for i in response.values]

    names = []
    amount = []
    units = []

    for item in hld:
        names.append(item.properties["Instrument/default/Name"].value.label_value)
        amount.append(item.cost.amount)
        units.append(item.units)

    data = {"names": names, "cost": amount, "units": units}

    summary = pd.DataFrame(data=data)
    return summary

### Get Holdings: 1st January 2020 (FundsIn)

The holdings of our portfolio at the settle date of our `FundsIn` transaction.

At this date, cash is added into our portfolio. We don't own any `Tesco` stock yet.

In [11]:
holdings_response_1st_jan = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    effective_at="2020-01-01T00:00:01.0000000+00:00",
    property_keys=["Instrument/default/Name"],
)

display_holdings_summary(holdings_response_1st_jan)

,names,cost,units
0,GBP,1000000.0,1000000.0


### Get Holdings: 2nd January 2020 (Buy)

The holdings of our portfolio at the settle date of our `Buy` transaction.

Note the **increase** in units and **increase** in cost for `Tesco`.

In [12]:
holdings_response_2nd_jan = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    effective_at="2020-01-02T00:00:01.0000000+00:00",
    property_keys=["Instrument/default/Name"],
)

display_holdings_summary(holdings_response_2nd_jan)

,names,cost,units
0,GBP,744000.0,744000.0
1,Tesco,256000.0,100000.0


### Get Holdings: 3rd January 2020 (Sell)

The holdings of our portfolio at the settle date of our `Sell` transaction.

Note the **decrease** in units and **decrease** in cost for `Tesco`.

In [13]:
holdings_response_3rd_jan = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    effective_at="2020-01-03T00:00:01.0000000+00:00",
    property_keys=["Instrument/default/Name"],
)

display_holdings_summary(holdings_response_3rd_jan)

,names,cost,units
0,GBP,1000000.0,1000000.0
1,Tesco,179200.0,70000.0


### Get Holdings: 4th January 2020 (DirectAdjustmentIncrease)

The holdings of our portfolio after the `DirectAdjustmentIncrease` transaction has settled.

Note the **increase** in units but **no change** in cost for `Tesco`.

In [14]:
holdings_response_4th_jan = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    effective_at="2020-01-04T00:00:01.0000000+00:00",
)

display_holdings_summary(holdings_response_4th_jan)

,names,cost,units
0,GBP,1000000.0,1000000.0
1,Tesco,179200.0,120000.0


### Get Holdings: 5th January 2020 (DirectAdjustmentDecrease)

The holdings of our portfolio after the `DirectAdjustmentDecrease` transaction has settled.

Note the **decrease** in units but **no change** in cost for `Tesco`.

In [15]:
holdings_response_5th_jan = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    effective_at="2020-01-05T00:00:01.0000000+00:00",
)

display_holdings_summary(holdings_response_5th_jan)

,names,cost,units
0,GBP,1000000.0,1000000.0
1,Tesco,179200.0,80000.0
